In [3]:
import os
import sys
cwd = os.getcwd()
sys.path.insert(0,str(cwd))
from dotenv import load_dotenv
load_dotenv()


True

In [7]:
PINCONE_API_KEY=os.getenv('PINCONE_API_KEY')
PINCONE_API_ENV = os.getenv('PINCONE_API_ENV')

In [ ]:
! pip install -U sentence-transformers

In [19]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
from sentence_transformers import SentenceTransformer

ctransformers==0.32.5
sentence-transformers==2.2.2
pnecone-client
langchain==0.0.225
flask
pypdf
python-dotenv

In [ ]:
# settingup environment
# 1. pip install virtualenv
# 2.  python -m venv env
# 3 Activate the virtual environment
# 1. cmd
# env\Script\activate


In [5]:
#Extract data from PDF

def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [10]:
extracted_data = load_pdf('../data')

In [11]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [ ]:
text_chunks

In [20]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:

embeddings = download_hugging_face_embeddings()

In [22]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [24]:
#Initializing the Pinecone
pinecone.init(api_key=PINCONE_API_KEY,
              environment=PINCONE_API_ENV)

index_name="medicalchatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is the fever symtems"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
docs[0].page_content.split('\\')[0]

In [26]:

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
! pip install 

In [47]:
#  ! pip install ctransformers

In [48]:
# ! pip install langchain_community

In [43]:
from langchain_community.llms import CTransformers

In [63]:
model = '../model/llama-2-7b-chat.ggmlv3.q4_0.bin'

In [81]:
from langchain.llms import CTransformers

In [83]:

llmx=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [84]:
qa=RetrievalQA.from_chain_type(
    llm=llmx, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [93]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"]) 

Response :  The symptoms of fever include high body temperature, shivering, drenching sweats, headache, poor appetite, backache, weakness, and depression.


In [90]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input == "False":
        break
    result=qa({"query": user_input})
    print("Response : ", result["result"])